In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from IPython.display import display, HTML
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
df = pd.read_pickle('df_model_2.pkl')
df.head()

total_sales platform  release_month  year         publisher  \
0  238000000.0      All             11  2011            Mojang   
1  175000000.0      All              9  2013    Rockstar Games   
0   82900000.0      Wii             11  2006          Nintendo   
2   70000000.0      All             12  2017  PUBG Corporation   
1   52000000.0       NS              4  2017          Nintendo   

          developer  meta_score  meta_nb_user_critics  meta_user_score  \
0         Mojang AB        93.0                7866.0             81.0   
1    Rockstar North        97.0                9212.0             83.0   
0      Nintendo EAD        76.0                 543.0             81.0   
2  PUBG Corporation        86.0                1994.0             48.0   
1      Nintendo EPD        92.0                2701.0             86.0   

                 game_collection  nb_platforms  genre_adventure  genre_arcade  \
0                      Minecraft           3.0             True         False   
1               Grand Theft Auto           5.0             True         False   
0                     Wii Sports           1.0            False         False   
2  playerunknown s battlegrounds           4.0            False         False   
1                     Mario Kart           1.0            False          True   

   genre_card-and-board-game  genre_fighting  genre_hack-and-slash-beat-em-up  \
0                      False           False                            False   
1                      False           False                            False   
0                      False           False                            False   
2                      False           False                            False   
1                      False           False                            False   

   genre_indie  genre_moba  genre_music  genre_pinball  genre_platform  \
0        False       False        False          False           False   
1        False       False        False          False           False   
0        False       False        False          False           False   
2        False       False        False          False           False   
1        False       False        False          False           False   

   genre_point-and-click  genre_puzzle  genre_quiz-trivia  genre_racing  \
0                  False         False              False         False   
1                  False         False              False         False   
0                  False         False              False         False   
2                  False         False              False         False   
1                  False         False              False          True   

   genre_real-time-strategy-rts  genre_role-playing-rpg  genre_shooter  \
0                         False                   False          False   
1                         False                   False           True   
0                         False                   False          False   
2                         False                   False           True   
1                         False                   False          False   

   genre_simulator  genre_sport  genre_strategy  genre_tactical  \
0             True        False           False           False   
1            False        False           False           False   
0             True         True           False           False   
2            False        False           False           False   
1            False        False           False           False   

   genre_turn-based-strategy-tbs  genre_visual-novel  mode_battle-royale  \
0                          False               False               False   
1                          False               False               False   
0                          False               False               False   
2                          False               False                True   
1                          False               False           

# NOTA: 

Le dataframe df est une base de données déjà traitée qui ne contient pas de valeurs manquantes. Cela signifie que toutes les valeurs des variables ont été traitées et qu'il n'y a pas d'entrées vides ou manquantes dans les colonnes du dataframe.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17470 entries, 0 to 19983
Data columns (total 51 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   total_sales                      17470 non-null  float64
 1   platform                         17470 non-null  object 
 2   release_month                    17470 non-null  int64  
 3   year                             17470 non-null  int32  
 4   publisher                        17470 non-null  object 
 5   developer                        17470 non-null  object 
 6   meta_score                       17470 non-null  float64
 7   meta_nb_user_critics             17470 non-null  float64
 8   meta_user_score                  17470 non-null  float64
 9   game_collection                  17470 non-null  object 
 10  nb_platforms                     17470 non-null  float64
 11  genre_adventure                  17470 non-null  bool   
 12  genre_arcade      

In [3]:
# affichage des valeurs manquantes
df.isna().sum()

total_sales                        0
platform                           0
release_month                      0
year                               0
publisher                          0
developer                          0
meta_score                         0
meta_nb_user_critics               0
meta_user_score                    0
game_collection                    0
nb_platforms                       0
genre_adventure                    0
genre_arcade                       0
genre_card-and-board-game          0
genre_fighting                     0
genre_hack-and-slash-beat-em-up    0
genre_indie                        0
genre_moba                         0
genre_music                        0
genre_pinball                      0
genre_platform                     0
genre_point-and-click              0
genre_puzzle                       0
genre_quiz-trivia                  0
genre_racing                       0
genre_real-time-strategy-rts       0
genre_role-playing-rpg             0
g

In [15]:
# Séparation du jeu de données en un ensemble d'entraînement et un ensemble de test

X = df.drop("total_sales", axis=1)
y = df['total_sales']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
cat_cols = ['platform', 'publisher', 'platform_name', 'platform_family', 'platform_category', 'platform_generation']   
num_cols = ['release_month', 'year', 'meta_score', 'meta_nb_user_critics', 'meta_user_score', 'nb_platforms', 'genre_adventure', 
            'genre_platform', 'genre_role-playing-rpg', 'genre_shooter', 'genre_strategy', 'genre_visual-novel', 'mode_battle-royale',
            'mode_co-operative', 'mode_mmo', 'mode_multiplayer', 'mode_single-player', 'mode_split-screen', 'perspective_first-person',
            'perspective_text', 'perspective_text', 'perspective_text', 'perspective_third-person', 'perspective_virtual-reality', 
            'genre_point-and-click', 'genre_fighting']
            
cat_train = X_train[cat_cols]
cat_test = X_test[cat_cols]
num_train, num_test = X_train[num_cols], X_test[num_cols]


In [16]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, drop="first", handle_unknown="ignore")
cat_train_encoded = ohe.fit_transform(cat_train)
cat_test_encoded = ohe.transform(cat_test) 

C:\Users\Amara\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Amara\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Vérification de la reussite de l'encodage

In [17]:
# Vérification des dimensions
print("Dimensions de cat_train_encoded:", cat_train_encoded.shape)
print("Dimensions de cat_test_encoded:", cat_test_encoded.shape)

# Catégories apprises
print("Catégories apprises:", ohe.categories_)

# Exemples de valeurs encodées
print("Exemples de valeurs encodées dans cat_train_encoded:")
print(cat_train_encoded[:5])
print("Exemples de valeurs encodées dans cat_test_encoded:")
print(cat_test_encoded[:5])

Dimensions de cat_train_encoded: (13976, 961)
Dimensions de cat_test_encoded: (3494, 961)
Catégories apprises: [array(['2600', '3DO', '3DS', 'All', 'Amig', 'DC', 'DS', 'GB', 'GBA',
       'GBC', 'GC', 'GEN', 'Int', 'N64', 'NES', 'NG', 'NS', 'OSX', 'PC',
       'PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSN', 'PSP', 'PSV', 'SAT',
       'SCD', 'SNES', 'WS', 'Wii', 'WiiU', 'X360', 'XB', 'XBL', 'XOne',
       'iOS'], dtype=object), array(['10TACLE Studios', '11 bit studios', '14 Degrees East', '17-Bit',
       '1C Company', '20th Century Fox Video Games', '2K Games',
       '2K Play', '2K Sports', '3DO', '3f Interactive', '505 Games',
       '5pb', '704Games', '7G//AMES', '7th Level', '8-4', '989 Sports',
       '989 Studios', 'A.Y.std', 'A1 Games', 'AGM Playism', 'AIA',
       'ANPA.US', 'AQ Interactive', 'ARUSH Entertainment', 'ASC Games',
       'ASCII Entertainment', 'ASCII Media Works', 'Abbey Games',
       'Access Software', 'Acclaim Entertainment', 'Accolade',
       'Acid Wizard Studio',

In [18]:
# Concaténation des données numériques et catégorielles encodées
X_train = np.concatenate((num_train, cat_train_encoded), axis=1)
X_test = np.concatenate((num_test, cat_test_encoded), axis=1)

print("Dimensions du jeu d'entraînement:", X_train.shape)
print("Dimensions du jeu de test:", X_test.shape)

Dimensions du jeu d'entraînement: (13976, 987)
Dimensions du jeu de test: (3494, 987)


### Modélisation

Rappel:

MSE = (1/n) * Σ(y - ŷ)²

Un MSE faible indiquent que les prédictions du modèle sont généralement proches des vraies valeurs de la variable cible.

Un MSE élevé indique que les prédictions du modèle sont éloignées des vraies valeurs.

le R² : est une mesure qui indique dans quelle mesure les variations de la variable dépendante peuvent être expliquées par les variations de la variable indépendante.

# Modéle 1: Regression Linéare



In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Mise à l'échelle des variables numériques
scaler = StandardScaler()
num_train_scaled = scaler.fit_transform(num_train)
num_test_scaled = scaler.transform(num_test)

# Concaténation des variables numériques et catégorielles
X_train = np.concatenate([num_train_scaled, cat_train_encoded], axis=1)
X_test = np.concatenate([num_test_scaled, cat_test_encoded], axis=1)

# Création et entraînement du modèle de régression linéaire
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

y_pred = regression_model.predict(X_test)

# Evaluation du modéle
from sklearn.metrics import r2_score, mean_squared_error

# Affichage des scores
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

Coefficient de détermination (R²) : -1.1399503696581325e+19
Erreur quadratique moyenne (MSE) : 7.564447608308994e+31
Erreur quadratique moyenne (RMSE) : 8697383289420442.0


# Analyse 

Le coefficient de détermination (R²) de -1.1399503696581325e+19 est une valeur anormalement basse et inhabituelle. Cela peut indiquer une très mauvaise performance du modèle de régression, voire une inadéquation complète entre le modèle et les données. Une valeur négative de R² suggère que le modèle est très inapproprié pour expliquer la variation des données. 

# Modéle 2 : RandomForestRegressor



In [9]:
from sklearn.ensemble import RandomForestRegressor

X_train = np.concatenate([num_train_scaled, cat_train_encoded], axis=1)
X_test = np.concatenate([num_test_scaled, cat_test_encoded], axis=1)

# Création et entraînement du modèle 
random_forest = RandomForestRegressor(random_state = 42)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

# Affichage des scores
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

Coefficient de détermination (R²) : 0.19955249614317117
Erreur quadratique moyenne (MSE) : 5311584931493.599
Erreur quadratique moyenne (RMSE) : 2304687.599544372


# Analyse :

les résultats de la pipeline montrent que le modèle de régression a une performance limitée. Le coefficient de détermination relativement bas suggère que le modèle n'explique qu'une petite partie de la variation des données. De plus, le MSE élevé et le RMSE indiquent une dispersion des erreurs et une précision limitée dans les prédictions du modèle. Il serait utile d'explorer d'autres modèles ou d'ajuster les caractéristiques utilisées pour améliorer les performances et réduire les erreurs.

# Pipline : RandomForestRegressor

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

cat_cols = ['platform', 'publisher', 'platform_name', 'platform_family', 'platform_category', 'platform_generation']   
num_cols = ['release_month', 'year', 'meta_score', 'meta_nb_user_critics', 'meta_user_score', 'nb_platforms', 'genre_adventure', 
            'genre_platform', 'genre_role-playing-rpg', 'genre_shooter', 'genre_strategy', 'genre_visual-novel', 'mode_battle-royale',
            'mode_co-operative', 'mode_mmo', 'mode_multiplayer', 'mode_single-player', 'mode_split-screen', 'perspective_first-person',
            'perspective_text', 'perspective_text', 'perspective_text', 'perspective_third-person', 'perspective_virtual-reality', 
            'genre_point-and-click', 'genre_fighting']
            
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse=False, drop="first"))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
        ('num', num_transformer, num_cols)
    ])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

pipe.fit(X, y)

predictions = pipe.predict(df)

# Affichage des scores
r2 = r2_score(y, predictions)
mse = mean_squared_error(y, predictions)
rmse = np.sqrt(mse)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

C:\Users\Amara\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Coefficient de détermination (R²) : 0.894931173766933
Erreur quadratique moyenne (MSE) : 1077157005232.929
Erreur quadratique moyenne (RMSE) : 1037861.7466854287


# Analyse : 

les résultats de la pipeline montrent que le modèle de régression a une performance relativement bonne. Le coefficient de détermination élevé suggère que le modèle explique une grande partie de la variation des données. Cependant, l'erreur quadratique moyenne (MSE) indique une certaine dispersion des erreurs, et le RMSE montre une erreur moyenne relativement faible. 

# Modéle 2 : Regression par arbre de décision

In [23]:
from sklearn.tree import DecisionTreeRegressor

# Création et entraînement du modèle d'Arbre de Décision
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = tree.predict(X_test)

# Affichage les scores
rmse = np.sqrt(mse)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

Coefficient de détermination (R²) : 0.08427871550837718
Erreur quadratique moyenne (MSE) : 6076515140240.412
Erreur quadratique moyenne (RMSE) : 5766123.172924323


# analyse :

les résultats montrent que le modèle de régression performe très mal. Le coefficient de détermination négatif et le MSE élevé indiquent que le modèle ne parvient pas à expliquer la variation des données de manière significative.

# Pipeline:Régression par arbre de décision 


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor


# On définit les colonnes catégorielles et numériques
cat_cols = ['platform', 'publisher', 'platform_name', 'platform_family', 'platform_category', 'platform_generation']   
num_cols = ['release_month', 'year', 'meta_score', 'meta_nb_user_critics', 'meta_user_score', 'nb_platforms', 'genre_adventure', 
            'genre_platform', 'genre_role-playing-rpg', 'genre_shooter', 'genre_strategy', 'genre_visual-novel', 'mode_battle-royale',
            'mode_co-operative', 'mode_mmo', 'mode_multiplayer', 'mode_single-player', 'mode_split-screen', 'perspective_first-person',
            'perspective_text', 'perspective_text', 'perspective_text', 'perspective_third-person', 'perspective_virtual-reality', 
            'genre_point-and-click', 'genre_fighting']
            

# On crée les transformateurs pour l'imputation et l'encodage
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse=False, drop="first"))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# On crée la colonne transformer pour appliquer les transformations aux colonnes appropriées
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
        ('num', num_transformer, num_cols)
    ])

# On crée la pipeline complète avec le modèle de régression
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor())
])

# On entraîne la pipeline
pipe.fit(X, y)

# On effectue les prédictions
predictions = pipe.predict(df)

# Calcul du coefficient de détermination (R²)
r2 = r2_score(y, predictions)

# Calcul de l'erreur quadratique moyenne (MSE)
mse = mean_squared_error(y, predictions)

# Calculez la racine carrée de la moyenne des carrés des différences (RMSE)
rmse = np.sqrt(mse)

# Affichage des scores
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

C:\Users\Amara\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Coefficient de détermination (R²) : 0.9999383613190699
Erreur quadratique moyenne (MSE) : 631914710.9330281
Erreur quadratique moyenne (RMSE) : 25137.913814257303


# Analyse 
Le coefficient de détermination (R²) élevé de 0,99 indique que la pipline par arbre de décision utilisée, est capable d'expliquer une grande partie de la variance des données cibles. En d'autres termes, environ 99,99 % de la variation observée dans les valeurs cibles peut être expliquée par les variables d'entrée utilisées dans le modèle. Cela suggère que le modèle est très performant pour prédire les valeurs cibles.



# Modéle 4: KNeighborsRegressor

KNeighborsRegressor: Le modèle KNeighborsRegressor, également connu sous le nom de "régression des k plus proches voisins", est une technique d'apprentissage automatique utilisée pour prédire des valeurs continues. Contrairement à la régression linéaire qui cherche à établir une relation linéaire entre les variables, la régression des k plus proches voisins est basée sur une approche non paramétrique.



In [11]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Création et entraînement du modèle KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = knn.predict(X_test)


# Affichage des scores
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

Coefficient de détermination (R²) : 0.2637166995607326
Erreur quadratique moyenne (MSE) : 4885806083571.838
Erreur quadratique moyenne (RMSE) : 2210385.9580561575


# Analyse : 

les résultats montrent que le modèle de régression a une performance modérée. Le coefficient de détermination indique que le modèle explique une partie significative de la variation des données, mais il y a encore une quantité importante de variation non expliquée. 

# Pipeline: KNeighborsRegressor

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor

# On définit les colonnes catégorielles et numériques
cat_cols = ['platform', 'publisher', 'platform_name', 'platform_family', 'platform_category', 'platform_generation']   
num_cols = ['release_month', 'year', 'meta_score', 'meta_nb_user_critics', 'meta_user_score', 'nb_platforms', 'genre_adventure', 
            'genre_platform', 'genre_role-playing-rpg', 'genre_shooter', 'genre_strategy', 'genre_visual-novel', 'mode_battle-royale',
            'mode_co-operative', 'mode_mmo', 'mode_multiplayer', 'mode_single-player', 'mode_split-screen', 'perspective_first-person',
            'perspective_text', 'perspective_text', 'perspective_text', 'perspective_third-person', 'perspective_virtual-reality', 
            'genre_point-and-click', 'genre_fighting']
            
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse=False, drop="first"))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
        ('num', num_transformer, num_cols)
    ])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

pipe.fit(X, y)

predictions = pipe.predict(df)


# Affichage des scores
r2 = r2_score(y, predictions)
mse = mean_squared_error(y, predictions)
rmse = np.sqrt(mse)
print("Coefficient de détermination (R²) :", r2)
print("Erreur quadratique moyenne (MSE) :", mse)
print("Erreur quadratique moyenne (RMSE) :", rmse)

C:\Users\Amara\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Coefficient de détermination (R²) : 0.33356835247182526
Erreur quadratique moyenne (MSE) : 6832202693989.696
Erreur quadratique moyenne (RMSE) : 2613848.253818438


# Analyse :

les résultats de la pipeline montrent que le modèle de régression a une performance modérée. Le coefficient de détermination indique que le modèle explique une partie significative de la variation des données, mais il y a encore une quantité importante de variation non expliquée

# Conclusion:

D'après l'évaluation des quatre modèles de machine learning appliqués, bien que les résultats soient relativement bas, le modèle KNeighborsRegressor avec un coefficient de détermination (R²) de 0.26 se démarque comme le meilleur.
